# Create one (1) area-weighted forcing file
We need to find how the ERA5 gridded forcing maps onto the catchment to create area-weighted forcing as SUMMA input. This involves two steps:
1. Intersect the ERA5 shape with the user's catchment shape to find the overlap between a given (sub) catchment and the forcing grid;
2. Create an area-weighted, catchment-averaged forcing time series.

The CANDEX package (https://github.com/ShervanGharari/candex_newgen) provides the necessary functionality to do this. CANDEX performs the GIS step (1, shapefile intersection) and the area-weighting step (2, create new forcing `.nc` files) as part of a single `run_candex()` call. To allow for parallelization, CANDEX can save the output from the GIS step into a restart `.csv` file which can be used to skip the GIS step. This allows (manual) parallelization of area-weighted forcing file generation after the GIS procedures have been run once. The full workflow here is thus:
1. [This script] Call `run_candex()` with ERA5 and user's shapefile, and one ERA5 forcing `.nc` file;
    - CANDEX performs intersection of both shapefiles;
    - CANDEX saves the outcomes of this intersection to a `.csv` file;
    - CANDEX creates an area-weighted forcing file from a single provided ERA5 source `.nc` file
2. [Follow-up script] Call `run_candex()` with intersection `.csv` file and all other forcing `.nc` files.
3. [Follow-up script] Apply lapse rates to temperature variable.

Parallelization of step 2 (2nd `run_candex()` call) requires an external loop that sends (batches of) the remaining ERA5 raw forcing files to individual processors. As with other steps that may be parallelized, creating code that does this is left to the user.



In [1]:
# modules
import os
import glob
import candex
from pathlib import Path
from shutil import rmtree
from shutil import copyfile
from datetime import datetime

#### Control file handling



In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_control_files')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find location of shapefiles

In [6]:
# Catchment shapefile path & name
catchment_path = read_from_control(controlFolder/controlFile,'intersect_dem_path')
catchment_name = read_from_control(controlFolder/controlFile,'intersect_dem_name')

In [7]:
# Specify default path if needed
if catchment_path == 'default':
    catchment_path = make_default_path('shapefiles/catchment_intersection/with_dem') # outputs a Path()
else:
    catchment_path = Path(catchment_path) # make sure a user-specified path is a Path()

In [8]:
# Forcing shapefile path & name
forcing_shape_path = read_from_control(controlFolder/controlFile,'forcing_shape_path')
forcing_shape_name = read_from_control(controlFolder/controlFile,'forcing_shape_name')

In [9]:
# Specify default path if needed
if forcing_shape_path == 'default':
    forcing_shape_path = make_default_path('shapefiles/forcing') # outputs a Path()
else:
    forcing_shape_path = Path(forcing_shape_path) # make sure a user-specified path is a Path()

#### Find where the intersection needs to go

In [10]:
# Intersected shapefile path. Name is set by CANDEX as [prefix]_intersected_shapefile.shp
intersect_path = read_from_control(controlFolder/controlFile,'intersect_forcing_path')

In [11]:
# Specify default path if needed
if intersect_path == 'default':
    intersect_path = make_default_path('shapefiles/catchment_intersection/with_forcing') # outputs a Path()
else:
    intersect_path = Path(intersect_path) # make sure a user-specified path is a Path()

In [12]:
# Make the folder if it doesn't exist
intersect_path.mkdir(parents=True, exist_ok=True)

#### Find the forcing files (merged ERA5 data)

In [13]:
# Location of merged ERA5 files
forcing_merged_path = read_from_control(controlFolder/controlFile,'forcing_merged_path')

In [14]:
# Specify default path if needed
if forcing_merged_path == 'default':
    forcing_merged_path = make_default_path('forcing/2_merged_data') # outputs a Path()
else:
    forcing_merged_path = Path(forcing_merged_path) # make sure a user-specified path is a Path()

In [15]:
# Find files in folder
forcing_files = [forcing_merged_path/file for file in os.listdir(forcing_merged_path) if os.path.isfile(forcing_merged_path/file)]

In [16]:
# Sort the files
forcing_files.sort()

#### Find where the temporary CANDEX files need to go

In [17]:
# Location for CANDEX temporary storage
forcing_candex_path = read_from_control(controlFolder/controlFile,'forcing_candex_path')

In [18]:
# Specify default path if needed
if forcing_candex_path == 'default':
    forcing_candex_path = make_default_path('forcing/3_temp_candex') # outputs a Path()
else:
    forcing_candex_path = Path(forcing_candex_path) # make sure a user-specified path is a Path()

In [19]:
# Make the folder if it doesn't exist
forcing_candex_path.mkdir(parents=True, exist_ok=True)

#### Find where the area-weighted forcing needs to go

In [20]:
# Location for CANDEX forcing output
forcing_basin_path = read_from_control(controlFolder/controlFile,'forcing_basin_avg_path')

In [21]:
# Specify default path if needed
if forcing_basin_path == 'default':
    forcing_basin_path = make_default_path('forcing/3_basin_averaged_data') # outputs a Path()
else:
    forcing_basin_path = Path(forcing_basin_path) # make sure a user-specified path is a Path()

In [22]:
# Make the folder if it doesn't exist
forcing_basin_path.mkdir(parents=True, exist_ok=True)

#### CANDEX

In [23]:
# Initialize a CANDEX object
cndx = candex.candex()

In [24]:
# Author name
cndx.author_name = 'SUMMA public workflow scripts'

In [25]:
# Data license
cndx.license = 'Copernicus data use license: https://cds.climate.copernicus.eu/api/v2/terms/static/licence-to-use-copernicus-products.pdf'

In [26]:
# Case name, used in CANDEX-generated file naes
cndx.case_name = read_from_control(controlFolder/controlFile,'domain_name')

In [27]:
# ERA5 shapefile and variable names
# Variable names can be hardcoded because we set them when we generate this shapefile as part of the workflow
cndx.source_shp     = forcing_shape_path/forcing_shape_name # shapefile
cndx.source_shp_lat = 'lat'                                 # name of the latitude field
cndx.source_shp_lon = 'lon'                                 # name of the longitude field

In [28]:
# Catchment shapefile and variable names
cndx.sink_shp = catchment_path/catchment_name
cndx.sink_shp_ID  = read_from_control(controlFolder/controlFile,'catchment_shp_hruid') # name of the HRU ID field
cndx.sink_shp_lat = read_from_control(controlFolder/controlFile,'catchment_shp_lat')   # name of the latitude field
cndx.sink_shp_lon = read_from_control(controlFolder/controlFile,'catchment_shp_lon')   # name of the longitude field

In [29]:
# ERA5 netcdf file and variable names
cndx.source_nc = str(forcing_files[0]) # first file in the list; Path() to string
cndx.var_names = ['airpres',
                  'LWRadAtm',
                  'SWRadAtm',
                  'pptrate',
                  'airtemp',
                  'spechum',
                  'windspd'] # variable names of forcing data - hardcoded because we prescribe them during ERA5 merging
cndx.var_lat   = 'latitude'  # name of the latitude dimensions
cndx.var_lon   = 'longitude' # name of the longitude dimension
cndx.var_time  = 'time'      # name of the time dimension

In [30]:
# Temporary folder where the CANDEX-generated GIS files and remapping file will be saved
cndx.temp_dir = str(forcing_candex_path) + '/' # Path() to string; ensure the trailing '/' CANDEX wants

In [31]:
# Output folder where the catchment-averaged forcing will be saved
cndx.output_dir = str(forcing_basin_path) + '/' # Path() to string; ensure the trailing '/' CANDEX wants

In [32]:
# Netcdf settings
cndx.remapped_dim_id = 'hru'     # name of the non-time dimension; prescribed by SUMMA
cndx.remapped_var_id = 'hruId'   # name of the variable associated with the non-time dimension
cndx.format_list     = ['f4']    # variable type to save forcing as. Single entry here will be used for all variables
cndx.fill_value_list = ['-9999'] # fill value

In [33]:
# Flag that we do not want the data stored in .csv in addition to .nc
cndx.save_csv  = False

In [34]:
# Flag that we currently have no remapping file
cndx.remap_csv = ''   

In [35]:
# Run candex
# Note on centroid warnings: in this case we use a regular lat/lon grid to represent ERA5 forcing and ...
#     centroid estimates without reprojecting are therefore acceptable.
# Note on deprecation warnings: this is a CANDEX issue that cannot be resolved here. Does not affect current use.
cndx.run_candex()

candex is given multiple varibales to be remapped but only on format and fill valuecandex repeat the format and fill value for all the variables in output files
candex detects that target shapefile is in WGS84 (epsg:4326)
candex detects that the field for ID is provided in sink/target shapefile
candex detects that the field latitude is provided in sink/target shapefile
candex detects that the field longitude is provided in sink/target shapefile
it seems everything is OK with the sink/target shapefile; added to candex object sink_shp_gpd
candex will save standard shapefile for candex claculation as:
C:\Globus endpoint\summaWorkflow_data\domain_BowAtBanff\forcing\3_temp_candex/BowAtBanff_sink_shapefile.shp
candex detects that the varibales from the netCDF files are identicalin dimensions of the varibales and latitude and longitude
candex detects that all the varibales have dimensions of:
['time', 'latitude', 'longitude']
candex detects that the longitude varibales has dimensions of:
['lo

C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex\lib\site-packages\candex\candex.py:109: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_2 ['lat_temp'] = shp_2.centroid.y
C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex\lib\site-packages\candex\candex.py:110: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_2 ['lon_temp'] = shp_2.centroid.x
C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis_candex\lib\site-packages\candex\candex.py:893: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  pairs = gpd.GeoDataFrame(nei, columns=['idx1','idx2'], crs=df1.crs)
C:\Users\wmk934\Anaconda3\envs\geospatialTools_qgis

------REMAPPING------
Remapping C:\Globus endpoint\summaWorkflow_data\domain_BowAtBanff\forcing\2_merged_data\ERA5_merged_197901.nc to C:\Globus endpoint\summaWorkflow_data\domain_BowAtBanff\forcing\3_basin_averaged_data/BowAtBanff_remapped_1979-01-01-00-00-00.nc
Started at date and time 2021-03-18 11:03:12.237417
Ended   at date and time 2021-03-18 11:03:42.227872
------


#### Move files to prescribed locations

In [36]:
# Remapping file 
remap_file = cndx.case_name + '_remapping.csv'
copyfile( cndx.temp_dir + remap_file, intersect_path / remap_file);

In [37]:
# Intersected shapefile
for file in glob.glob(cndx.temp_dir + cndx.case_name + '_intersected_shapefile.*'):
    copyfile( file, intersect_path / os.path.basename(file));

In [38]:
# Remove the temporary CANDEX directory to save space
try:
    rmtree(cndx.temp_dir)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))    

#### Code provenance - intersection shapefile
Generates a basic log file in the domain folder and copies the control file and itself there.

In [39]:
# Set the log path and file name
logPath = intersect_path
log_suffix = '_catchment_forcing_intersect_log.txt'

In [40]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [41]:
# Copy this script
thisFile = '1_make_one_weighted_forcing_file.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [42]:
# Get current date and time
now = datetime.now()

In [43]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Intersect shapefiles of catchment and ERA5.']
    for txt in lines:
        file.write(txt)  

#### Code provenance - weighted forcing file
Generates a basic log file in the domain folder and copies the control file and itself there.

In [44]:
# Set the log path and file name
logPath = forcing_basin_path
log_suffix = '_create_one_weighted_forcing_file_log.txt'

In [45]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [46]:
# Copy this script
thisFile = '1_make_one_weighted_forcing_file.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [47]:
# Get current date and time
now = datetime.now()

In [48]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Made a weighted forcing file based on intersect shapefiles of catchment and ERA5.']
    for txt in lines:
        file.write(txt)  